<a href="https://colab.research.google.com/github/myprefrontalcortex/cycling/blob/main/Advanced_Bike_Maintenance_Tracker_(V4_Feature_Enhancements).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import json
import os
import datetime
import csv

# --- Constants ---
DEFAULT_HOT_WAX_INTERVAL = 1500  # km
DEFAULT_DRIP_WAX_INTERVAL = 150  # km
DEFAULT_SEALANT_CHECK_DAYS = 90 # days
YELLOW_THRESHOLD_KM = 300   # km for hot wax warning
YELLOW_THRESHOLD_DAYS = 15 # days for sealant warning

DEFAULT_DATA_FILENAME = 'advanced_bike_maintenance_data.json'
CONFIG_FILENAME = 'tracker_config.json'

class AdvancedBikeMaintenanceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Bike Maintenance Tracker")
        self.data_file_path = self._get_data_file_path()
        if not self.data_file_path:
            messagebox.showerror("Configuration Error", "Could not set up data file path. Exiting.")
            self.root.destroy()
            return

        window_width = 850
        window_height = 610
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        position_x = (screen_width // 2) - (window_width // 2)
        position_y = (screen_height // 2) - (window_height // 2) - 50
        if position_y < 20: position_y = 20
        self.root.geometry(f"{window_width}x{window_height}+{position_x}+{position_y}")

        self.style = ttk.Style()
        self.default_font_family = "Arial"
        self.default_font_size = 10
        self.default_font = (self.default_font_family, self.default_font_size)
        self.heading_font = (self.default_font_family, self.default_font_size + 1, "bold")
        self.small_font = (self.default_font_family, self.default_font_size -1)


        self.style.configure('.', font=self.default_font)
        self.style.configure('TNotebook.Tab', font=self.heading_font, padding=[10, 5])
        self.style.configure('TLabelframe.Label', font=self.heading_font)
        self.style.configure("Treeview.Heading", font=self.heading_font)
        self.style.configure("Treeview", font=self.default_font, rowheight=int(self.default_font_size * 2.2))

        self.colors = {
            "bg_root": "#F0F0F0", "bg_notebook_tab_content": "#FFFFFF",
            "bg_odo_frame": "#E6EBF0", "bg_chains_frame": "#E6F0E6",
            "bg_components_frame": "#F0EAE6", "bg_sealant_frame": "#F0E6F0",
            "bg_log_frame": "#FFFFFF", "bg_settings_frame": "#FAF0E6"
        }
        self.root.configure(bg=self.colors["bg_root"])
        self.data = self._load_all_data()

        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill='both', padx=10, pady=5)
        self.notebook.update_idletasks()

        self.log_frame = ttk.Frame(self.notebook, padding="10")
        self.notebook.add(self.log_frame, text="Maintenance Log")
        self._create_log_panel(self.log_frame)

        self.export_settings_frame = ttk.Frame(self.notebook, padding="10")
        self.notebook.add(self.export_settings_frame, text="Import/Export/Settings")
        self._create_export_settings_panel(self.export_settings_frame)

        self.bike_tabs_widgets = {}
        self._create_all_bike_tabs()
        self._set_default_tab()
        self.root.protocol("WM_DELETE_WINDOW", self._on_closing)
        self._update_all_displays()

    def _get_data_file_path(self):
        config_path = CONFIG_FILENAME
        data_path = None
        if os.path.exists(config_path):
            try:
                with open(config_path, 'r') as f:
                    config_data = json.load(f)
                    data_path = config_data.get("data_file_location")
                if data_path and os.path.exists(data_path) and data_path.endswith(DEFAULT_DATA_FILENAME): return data_path
                elif data_path and os.path.isdir(os.path.dirname(data_path)) and data_path.endswith(DEFAULT_DATA_FILENAME): return data_path
            except (IOError, json.JSONDecodeError): pass
        messagebox.showinfo("Data File Setup", f"Please locate your '{DEFAULT_DATA_FILENAME}' data file or choose a location to create it.")
        if messagebox.askyesno("Data File", f"Do you have an existing '{DEFAULT_DATA_FILENAME}' file to open?"):
            chosen_path = filedialog.askopenfilename(title=f"Select '{DEFAULT_DATA_FILENAME}'", initialfile=DEFAULT_DATA_FILENAME, defaultextension=".json", filetypes=[("JSON data files", "*.json"), ("All files", "*.*")])
        else:
            chosen_path = filedialog.asksaveasfilename(title=f"Create New '{DEFAULT_DATA_FILENAME}' in Synced Folder", initialfile=DEFAULT_DATA_FILENAME, defaultextension=".json", filetypes=[("JSON data files", "*.json"), ("All files", "*.*")])
        if chosen_path:
            if not chosen_path.endswith(DEFAULT_DATA_FILENAME):
                dir_name, file_name_only = os.path.split(chosen_path)
                chosen_path = os.path.join(dir_name, DEFAULT_DATA_FILENAME)
                messagebox.showinfo("Filename Correction", f"Data file will be saved as '{DEFAULT_DATA_FILENAME}' in the chosen directory.")
            try:
                with open(config_path, 'w') as f: json.dump({"data_file_location": chosen_path}, f, indent=4)
                return chosen_path
            except IOError: messagebox.showerror("Error", f"Could not save configuration to '{config_path}'."); return None
        else: messagebox.showwarning("Setup Cancelled", "Data file location not set."); return None

    def _create_scrollable_tab_content(self, parent_notebook, tab_text, bike_id_key_for_style):
        tab_outer_frame = ttk.Frame(parent_notebook) # Removed direct style for simplicity, can be added if needed
        parent_notebook.insert(0, tab_outer_frame, text=tab_text)
        canvas = tk.Canvas(tab_outer_frame, highlightthickness=0, bg=self.colors["bg_notebook_tab_content"])
        canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar = ttk.Scrollbar(tab_outer_frame, orient="vertical", command=canvas.yview)
        scrollbar.pack(side=tk.RIGHT, fill="y")
        canvas.configure(yscrollcommand=scrollbar.set)
        content_frame = ttk.Frame(canvas, padding="10")
        content_frame.configure(style=f"{bike_id_key_for_style}.InnerContent.TFrame") # Apply style to inner frame
        self.style.configure(f"{bike_id_key_for_style}.InnerContent.TFrame", background=self.colors["bg_notebook_tab_content"])
        canvas_frame_id = canvas.create_window((0, 0), window=content_frame, anchor="nw")
        def _on_frame_configure(event): canvas.configure(scrollregion=canvas.bbox("all"))
        def _on_canvas_configure(event): canvas.itemconfig(canvas_frame_id, width=event.width)
        content_frame.bind("<Configure>", _on_frame_configure)
        canvas.bind("<Configure>", _on_canvas_configure)
        return content_frame

    def _create_all_bike_tabs(self):
        existing_tabs = list(self.notebook.tabs()) # Get list of current tab identifiers (widget paths)
        for tab_id_path in existing_tabs:
            try:
                tab_text = self.notebook.tab(tab_id_path, "text")
                if tab_text not in ["Maintenance Log", "Import/Export/Settings"]:
                    self.notebook.forget(tab_id_path)
            except tk.TclError: continue
        self.bike_tabs_widgets = {}
        self.root.update_idletasks()

        bike_ids_ordered = list(self.data.get("bikes", {}).keys())
        for bike_id in reversed(bike_ids_ordered):
            bike_info = self.data["bikes"][bike_id]
            scrollable_content_area = self._create_scrollable_tab_content(self.notebook, bike_info.get("name", bike_id), bike_id)
            self.bike_tabs_widgets[bike_id] = self._create_bike_panel_widgets(scrollable_content_area, bike_id)

    def _set_default_tab(self):
        nebula_tab_id = None; first_bike_tab_id = None
        bike1_data = self.data.get("bikes", {}).get("bike1", {}); nebula_name = bike1_data.get("name", "Nebula")
        for i in range(self.notebook.index("end")):
            try:
                tab_content_frame_path = self.notebook.tabs()[i]
                tab_text = self.notebook.tab(tab_content_frame_path, "text")
                if tab_text == nebula_name: nebula_tab_id = tab_content_frame_path; break
                if first_bike_tab_id is None and tab_text not in ["Maintenance Log", "Import/Export/Settings"]: first_bike_tab_id = tab_content_frame_path
            except tk.TclError: continue
        if nebula_tab_id: self.notebook.select(nebula_tab_id)
        elif first_bike_tab_id: self.notebook.select(first_bike_tab_id)

    def _get_default_data_structure(self):
        now_str = datetime.datetime.now().strftime("%Y-%m-%d")
        return {
            "bikes": {
                "bike1": {
                    "name": "Nebula", "type": "di2_disc", "current_odo": 0.0,
                    "chains": [
                        {"id": "Nebula Chain A", "last_hot_wax_odo": 0.0, "last_drip_wax_odo": 0.0, "km_ridden_on_chain": 0.0, "is_active": True, "hot_wax_count": 0},
                        {"id": "Nebula Chain B", "last_hot_wax_odo": 0.0, "last_drip_wax_odo": 0.0, "km_ridden_on_chain": 0.0, "is_active": False, "hot_wax_count": 0},
                    ],
                    "components": {
                        "tires": {"last_replaced_odo": 0.0, "notes": ""},
                        "brake_pads_disc_front": {"last_replaced_odo": 0.0, "notes": ""}, # Split
                        "brake_pads_disc_rear": {"last_replaced_odo": 0.0, "notes": ""},  # Split
                        "cassette": {"last_replaced_odo": 0.0, "notes": ""},
                        "chainrings": {"last_replaced_odo": 0.0, "notes": ""},
                    },
                    "sealant": {"last_checked_date": now_str, "notes": ""},
                },
                "bike2": {
                    "name": "Felt f95", "type": "mechanical_rim", "current_odo": 0.0,
                    "chains": [
                        {"id": "Felt Chain", "last_hot_wax_odo": 0.0, "last_drip_wax_odo": 0.0, "km_ridden_on_chain": 0.0, "is_active": True, "hot_wax_count": 0},
                    ],
                    "components": {
                        "tires": {"last_replaced_odo": 0.0, "notes": ""},
                        "brake_pads_rim": {"last_replaced_odo": 0.0, "notes": ""}, # Rim brakes for this bike
                        "cassette": {"last_replaced_odo": 0.0, "notes": ""},
                        "chainrings": {"last_replaced_odo": 0.0, "notes": ""},
                    },
                }
            },
            "maintenance_log": []
        }

    def _create_bike_panel_widgets(self, parent_frame_for_content, bike_id):
        widgets = {}
        bike_data = self.data["bikes"][bike_id]

        odo_frame = ttk.LabelFrame(parent_frame_for_content, text="Bike Odometer", padding="10")
        odo_frame.config(style=f"{bike_id}.Odo.TLabelframe")
        odo_frame.pack(fill="x", padx=5, pady=5, anchor="n")
        ttk.Label(odo_frame, text="Current Bike Odo (km):", background=self.colors["bg_odo_frame"]).grid(row=0, column=0, padx=5, pady=5, sticky="w")
        widgets['current_odo_entry'] = ttk.Entry(odo_frame, width=15, font=self.default_font)
        widgets['current_odo_entry'].grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        widgets['current_odo_entry'].insert(0, str(bike_data.get("current_odo", 0.0)))
        ttk.Button(odo_frame, text="Update & Refresh Displays", command=lambda b_id=bike_id: self._update_bike_odometer_and_refresh(b_id)).grid(row=0, column=2, padx=10, pady=5)
        odo_frame.columnconfigure(1, weight=1)

        chains_frame = ttk.LabelFrame(parent_frame_for_content, text="Chains", padding="10")
        chains_frame.config(style=f"{bike_id}.Chains.TLabelframe")
        chains_frame.pack(fill="x", padx=5, pady=5, anchor="n")
        widgets['chain_frames_details'] = []

        for i, chain_info in enumerate(bike_data.get("chains", [])):
            chain_sub_frame = ttk.Frame(chains_frame, padding="5", style=f"{bike_id}.ChainDetail.TFrame")
            chain_sub_frame.pack(fill="x", pady=2)
            chain_detail_widgets = {}

            status_text = " (Active)" if chain_info.get("is_active") else " (Inactive)"
            chain_detail_widgets['status_label'] = ttk.Label(chain_sub_frame, text=f"{chain_info['id']}{status_text}", font=self.heading_font, background=self.colors["bg_chains_frame"])
            chain_detail_widgets['status_label'].grid(row=0, column=0, columnspan=5, sticky="w", pady=(5,2)) # Increased columnspan

            ttk.Label(chain_sub_frame, text="Km on this chain:", background=self.colors["bg_chains_frame"]).grid(row=1, column=0, sticky="w", padx=5)
            chain_detail_widgets['km_ridden_label'] = ttk.Label(chain_sub_frame, text=f"{chain_info.get('km_ridden_on_chain', 0.0):.0f} km", background=self.colors["bg_chains_frame"])
            chain_detail_widgets['km_ridden_label'].grid(row=1, column=1, sticky="w", padx=5)

            ttk.Label(chain_sub_frame, text="Hot Waxes:", background=self.colors["bg_chains_frame"]).grid(row=1, column=2, sticky="e", padx=(10,2))
            chain_detail_widgets['hot_wax_count_label'] = ttk.Label(chain_sub_frame, text=f"{chain_info.get('hot_wax_count',0)}", background=self.colors["bg_chains_frame"])
            chain_detail_widgets['hot_wax_count_label'].grid(row=1, column=3, sticky="w", padx=2)
            ttk.Button(chain_sub_frame, text="New Chain/Reset Km", command=lambda b=bike_id, c_idx=i: self._reset_chain_km(b, c_idx), style="Small.TButton").grid(row=1, column=4, padx=5, sticky="e")
            self.style.configure("Small.TButton", font=self.small_font)


            ttk.Label(chain_sub_frame, text="Last Hot Wax (Bike Odo):", background=self.colors["bg_chains_frame"]).grid(row=2, column=0, sticky="w", padx=5)
            chain_detail_widgets['edit_hot_wax'] = ttk.Entry(chain_sub_frame, width=10, font=self.default_font) # Adjusted width
            chain_detail_widgets['edit_hot_wax'].grid(row=2, column=1, sticky="ew", padx=5)
            chain_detail_widgets['edit_hot_wax'].insert(0, str(chain_info.get("last_hot_wax_odo", 0.0)))
            chain_detail_widgets['hot_wax_due_label'] = ttk.Label(chain_sub_frame, text="Due: N/A", background=self.colors["bg_chains_frame"])
            chain_detail_widgets['hot_wax_due_label'].grid(row=2, column=2, columnspan=2, sticky="w", padx=10) # Span to fill space
            ttk.Button(chain_sub_frame, text="Log Hot Wax", command=lambda b=bike_id, c_idx=i: self._log_chain_service(b, c_idx, "hot_wax")).grid(row=2, column=4, padx=5, sticky="e") # Moved to column 4

            ttk.Label(chain_sub_frame, text="Last Drip Wax (Bike Odo):", background=self.colors["bg_chains_frame"]).grid(row=3, column=0, sticky="w", padx=5)
            chain_detail_widgets['edit_drip_wax'] = ttk.Entry(chain_sub_frame, width=10, font=self.default_font) # Adjusted width
            chain_detail_widgets['edit_drip_wax'].grid(row=3, column=1, sticky="ew", padx=5)
            chain_detail_widgets['edit_drip_wax'].insert(0, str(chain_info.get("last_drip_wax_odo", 0.0)))
            chain_detail_widgets['drip_wax_due_label'] = ttk.Label(chain_sub_frame, text="Due: N/A", background=self.colors["bg_chains_frame"])
            chain_detail_widgets['drip_wax_due_label'].grid(row=3, column=2, columnspan=2, sticky="w", padx=10) # Span
            ttk.Button(chain_sub_frame, text="Log Drip Wax", command=lambda b=bike_id, c_idx=i: self._log_chain_service(b, c_idx, "drip_wax")).grid(row=3, column=4, padx=5, sticky="e") # Moved to column 4

            chain_sub_frame.columnconfigure(1, weight=1)
            chain_sub_frame.columnconfigure(2, weight=0) # Due label
            chain_sub_frame.columnconfigure(3, weight=0) # (empty for now or for count)
            chain_sub_frame.columnconfigure(4, weight=0) # Log buttons
            widgets['chain_frames_details'].append({"id": chain_info['id'], "ui": chain_detail_widgets})
            if i < len(bike_data.get("chains", [])) -1 :
                 ttk.Separator(chains_frame, orient='horizontal').pack(fill='x', pady=5, after=chain_sub_frame)

        if len(bike_data.get("chains", [])) > 1:
            widgets['swap_chain_button'] = ttk.Button(chains_frame, text="Swap Active Chain", command=lambda b_id=bike_id: self._swap_active_chain(b_id))
            widgets['swap_chain_button'].pack(pady=10)

        components_frame = ttk.LabelFrame(parent_frame_for_content, text="Other Components (Kilometer Based)", padding="10")
        components_frame.config(style=f"{bike_id}.Components.TLabelframe")
        components_frame.pack(fill="x", padx=5, pady=5, anchor="n")
        widgets['component_labels'] = {}

        component_map = {
            "tires": "Tires",
            "brake_pads_disc_front": "Disc Brake Pads (Front)",
            "brake_pads_disc_rear": "Disc Brake Pads (Rear)",
            "brake_pads_rim": "Rim Brake Pads",
            "cassette": "Cassette", "chainrings": "Chainrings"
        }
        row_idx = 0
        for comp_key, comp_name in component_map.items():
            if comp_key in bike_data.get("components", {}):
                ttk.Label(components_frame, text=f"{comp_name} - Km since replacement:", background=self.colors["bg_components_frame"]).grid(row=row_idx, column=0, sticky="w", padx=5, pady=3)
                label_widget = ttk.Label(components_frame, text="0 km", background=self.colors["bg_components_frame"])
                label_widget.grid(row=row_idx, column=1, sticky="w", padx=5)
                widgets['component_labels'][comp_key] = label_widget
                ttk.Button(components_frame, text=f"Log {comp_name} Replacement", command=lambda b_id=bike_id, c_key=comp_key, c_name=comp_name: self._log_component_replacement(b_id, c_key, c_name)).grid(row=row_idx, column=2, padx=10, pady=3, sticky="e")
                row_idx += 1
        components_frame.columnconfigure(1, weight=1)

        if "sealant" in bike_data:
            sealant_frame = ttk.LabelFrame(parent_frame_for_content, text="Tubeless Sealant", padding="10")
            sealant_frame.config(style=f"{bike_id}.Sealant.TLabelframe")
            sealant_frame.pack(fill="x", padx=5, pady=5, anchor="n")
            widgets['sealant_labels'] = {}
            ttk.Label(sealant_frame, text="Last Checked:", background=self.colors["bg_sealant_frame"]).grid(row=0, column=0, sticky="w", padx=5)
            widgets['sealant_labels']['last_checked_label'] = ttk.Label(sealant_frame, text=bike_data["sealant"].get("last_checked_date", "N/A"), background=self.colors["bg_sealant_frame"])
            widgets['sealant_labels']['last_checked_label'].grid(row=0, column=1, sticky="w", padx=5)
            widgets['sealant_labels']['due_label'] = ttk.Label(sealant_frame, text="Next Check: N/A", background=self.colors["bg_sealant_frame"])
            widgets['sealant_labels']['due_label'].grid(row=0, column=2, sticky="w", padx=10)
            ttk.Button(sealant_frame, text="Log Sealant Check", command=lambda b_id=bike_id: self._log_sealant_check(b_id)).grid(row=0, column=3, padx=10, sticky="e")
            sealant_frame.columnconfigure(1, weight=1); sealant_frame.columnconfigure(2, weight=1)
        return widgets

    def _update_bike_odometer_and_refresh(self, bike_id):
        if bike_id not in self.bike_tabs_widgets: return
        widgets = self.bike_tabs_widgets[bike_id]
        try:
            new_odo = float(widgets['current_odo_entry'].get())
            current_stored_odo = self.data["bikes"][bike_id].get("current_odo", 0.0)
            if new_odo < current_stored_odo:
                if not messagebox.askyesno("Odometer Warning", "New odometer is less than previous. Continue?"):
                    widgets['current_odo_entry'].delete(0, tk.END); widgets['current_odo_entry'].insert(0, str(current_stored_odo))
                    return
            odo_diff = new_odo - current_stored_odo
            if odo_diff > 0:
                for chain in self.data["bikes"][bike_id]["chains"]:
                    if chain["is_active"]: chain["km_ridden_on_chain"] = chain.get("km_ridden_on_chain", 0.0) + odo_diff; break
            self.data["bikes"][bike_id]["current_odo"] = new_odo
            self._save_all_data(); self._update_all_displays()
        except ValueError:
            messagebox.showerror("Input Error", "Current odometer must be a valid number.")
            widgets['current_odo_entry'].delete(0, tk.END); widgets['current_odo_entry'].insert(0, str(self.data["bikes"][bike_id].get("current_odo", 0.0)))

    def _swap_active_chain(self, bike_id):
        bike_data = self.data["bikes"][bike_id]
        if len(bike_data["chains"]) < 2: return
        current_bike_odo = bike_data.get("current_odo", 0.0)
        active_chain_index = next((i for i, chain in enumerate(bike_data["chains"]) if chain["is_active"]), -1)
        if active_chain_index == -1: messagebox.showerror("Chain Swap Error", "No active chain found."); return

        inactive_chain_index = (active_chain_index + 1) % len(bike_data["chains"])
        newly_activated_chain = bike_data["chains"][inactive_chain_index]

        if messagebox.askyesno("Hot Wax?", f"Did you also hot wax '{newly_activated_chain['id']}' before activating it?"):
            newly_activated_chain["last_hot_wax_odo"] = current_bike_odo
            newly_activated_chain["last_drip_wax_odo"] = current_bike_odo # Reset drip on hot wax
            newly_activated_chain["hot_wax_count"] = newly_activated_chain.get("hot_wax_count", 0) + 1
            self._add_log_entry(bike_id, f"{newly_activated_chain['id']} Hot Wax (on swap)", current_bike_odo, "Hot waxed during chain swap.")

        bike_data["chains"][active_chain_index]["is_active"] = False
        newly_activated_chain["is_active"] = True

        self._add_log_entry(bike_id, f"Chain Swap: {bike_data['chains'][active_chain_index]['id']} -> {newly_activated_chain['id']}", current_bike_odo, "Chain rotation performed.")
        self._save_all_data(); self._update_bike_panel_display(bike_id)
        messagebox.showinfo("Chain Swap", f"Active chain swapped to {newly_activated_chain['id']}.")

    def _reset_chain_km(self, bike_id, chain_index):
        bike_data = self.data["bikes"][bike_id]
        chain_data = bike_data["chains"][chain_index]
        if messagebox.askyesno("Confirm Reset", f"Reset kilometers for '{chain_data['id']}' (e.g., new chain installed)?\nThis will also log it as freshly hot waxed (count #1) at current bike odometer."):
            current_bike_odo = bike_data.get("current_odo", 0.0)
            chain_data["km_ridden_on_chain"] = 0.0
            chain_data["last_hot_wax_odo"] = current_bike_odo
            chain_data["last_drip_wax_odo"] = current_bike_odo
            chain_data["hot_wax_count"] = 1
            self._add_log_entry(bike_id, f"New Chain / Km Reset: {chain_data['id']}", current_bike_odo, "Km reset, assumed freshly hot waxed.")
            self._save_all_data(); self._update_bike_panel_display(bike_id)

    def _log_chain_service(self, bike_id, chain_index, service_type):
        if bike_id not in self.bike_tabs_widgets: return
        bike_data = self.data["bikes"][bike_id]; chain_data = bike_data["chains"][chain_index]
        chain_ui_widgets = self.bike_tabs_widgets[bike_id]['chain_frames_details'][chain_index]['ui']
        try:
            service_odo_str = chain_ui_widgets['edit_hot_wax'].get() if service_type == "hot_wax" else chain_ui_widgets['edit_drip_wax'].get()
            service_odo = float(service_odo_str)
            if service_odo > bike_data["current_odo"]: messagebox.showerror("Input Error", f"Service odo > current bike odo."); return

            prev_service_odo = chain_data.get(f"last_{service_type}_odo", 0.0)
            if service_odo < prev_service_odo: messagebox.showwarning("Input Warning", f"New {service_type.replace('_',' ')} odo < previous.")

            log_item_name = f"{chain_data['id']} {service_type.replace('_', ' ').title()}"
            chain_data[f"last_{service_type}_odo"] = service_odo
            if service_type == "hot_wax":
                chain_data["last_drip_wax_odo"] = service_odo # Reset drip on hot wax
                chain_data["hot_wax_count"] = chain_data.get("hot_wax_count", 0) + 1
                self._add_log_entry(bike_id, f"{chain_data['id']} Drip Wax (with Hot Wax)", service_odo, "Drip wax reset due to hot wax.")

            self._add_log_entry(bike_id, log_item_name, service_odo, f"Logged {service_type.replace('_', ' ')}.")
            self._save_all_data(); self._update_bike_panel_display(bike_id)
            messagebox.showinfo("Service Logged", f"{log_item_name} logged at {service_odo:.0f} km.")
        except ValueError: messagebox.showerror("Input Error", "Service odometer must be a valid number.")

    def _log_component_replacement(self, bike_id, component_key, component_name):
        bike_data = self.data["bikes"][bike_id]; current_bike_odo = bike_data.get("current_odo", 0.0)
        if component_key not in bike_data["components"]: messagebox.showerror("Error", f"Component {component_key} not found."); return
        bike_data["components"][component_key]["last_replaced_odo"] = current_bike_odo
        self._add_log_entry(bike_id, f"{component_name} Replacement", current_bike_odo, f"{component_name} replaced.")
        self._save_all_data(); self._update_bike_panel_display(bike_id)
        messagebox.showinfo("Replacement Logged", f"{component_name} replacement logged at {current_bike_odo:.0f} km.")

    def _log_sealant_check(self, bike_id):
        bike_data = self.data["bikes"][bike_id]; current_bike_odo = bike_data.get("current_odo", 0.0)
        today_str = datetime.datetime.now().strftime("%Y-%m-%d")
        if "sealant" not in bike_data: messagebox.showerror("Error", "Sealant tracking not for this bike."); return
        bike_data["sealant"]["last_checked_date"] = today_str
        self._add_log_entry(bike_id, "Sealant Check", current_bike_odo, f"Sealant checked on {today_str}.")
        self._save_all_data(); self._update_bike_panel_display(bike_id)
        messagebox.showinfo("Sealant Check Logged", f"Sealant check logged for {today_str}.")

    def _add_log_entry(self, bike_id, item_name, odo_at_service, notes=""):
        log_entry = {"date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "bike_name": self.data["bikes"][bike_id].get("name", bike_id), "item": item_name, "odo_at_service": f"{odo_at_service:.0f}", "notes": notes}
        self.data["maintenance_log"].insert(0, log_entry); self._update_log_display()

    def _update_all_displays(self):
        for bike_id in self.data.get("bikes", {}).keys():
            if bike_id in self.bike_tabs_widgets: self._update_bike_panel_display(bike_id)
        self._update_log_display()

    def _update_bike_panel_display(self, bike_id):
        if bike_id not in self.data["bikes"] or bike_id not in self.bike_tabs_widgets: return
        bike_data = self.data["bikes"][bike_id]; widgets = self.bike_tabs_widgets[bike_id]
        current_bike_odo = bike_data.get("current_odo", 0.0)
        widgets['current_odo_entry'].delete(0, tk.END); widgets['current_odo_entry'].insert(0, str(current_bike_odo))
        for i, chain_info in enumerate(bike_data.get("chains", [])):
            if i < len(widgets['chain_frames_details']):
                chain_ui = widgets['chain_frames_details'][i]['ui']
                status_text = " (Active)" if chain_info.get("is_active") else " (Inactive)"
                chain_ui['status_label'].config(text=f"{chain_info['id']}{status_text}")
                chain_ui['km_ridden_label'].config(text=f"{chain_info.get('km_ridden_on_chain', 0.0):.0f} km")
                chain_ui['hot_wax_count_label'].config(text=f"{chain_info.get('hot_wax_count',0)}")
                km_since_hot = -1.0; km_since_drip = -1.0
                if chain_info.get("is_active"):
                    km_since_hot = current_bike_odo - chain_info.get("last_hot_wax_odo", current_bike_odo)
                    km_since_drip = current_bike_odo - chain_info.get("last_drip_wax_odo", current_bike_odo)
                km_to_next_hot = DEFAULT_HOT_WAX_INTERVAL - km_since_hot if km_since_hot >=0 else float('inf')
                km_to_next_drip = DEFAULT_DRIP_WAX_INTERVAL - km_since_drip if km_since_drip >=0 else float('inf')
                hot_due_text = f"Due: {km_to_next_hot:.0f} km" if chain_info.get("is_active") else "Due: N/A (Inactive)"
                drip_due_text = f"Due: {km_to_next_drip:.0f} km" if chain_info.get("is_active") else "Due: N/A (Inactive)"
                chain_ui['hot_wax_due_label'].config(text=hot_due_text, foreground="red" if chain_info.get("is_active") and km_to_next_hot <= 0 else ("darkorange" if chain_info.get("is_active") and km_to_next_hot <= YELLOW_THRESHOLD_KM else "forest green"))
                chain_ui['drip_wax_due_label'].config(text=drip_due_text, foreground="red" if chain_info.get("is_active") and km_to_next_drip <= 0 else "forest green")
                chain_ui['edit_hot_wax'].delete(0, tk.END); chain_ui['edit_hot_wax'].insert(0, str(chain_info.get("last_hot_wax_odo", 0.0)))
                chain_ui['edit_drip_wax'].delete(0, tk.END); chain_ui['edit_drip_wax'].insert(0, str(chain_info.get("last_drip_wax_odo", 0.0)))
        for comp_key, label_widget in widgets.get('component_labels', {}).items():
            if comp_key in bike_data.get("components", {}):
                comp_data = bike_data["components"][comp_key]
                km_since_replacement = current_bike_odo - comp_data.get("last_replaced_odo", 0.0)
                label_widget.config(text=f"{km_since_replacement:.0f} km")
        if "sealant" in bike_data and "sealant_labels" in widgets:
            sealant_data = bike_data["sealant"]; sealant_ui = widgets['sealant_labels']
            sealant_ui['last_checked_label'].config(text=sealant_data.get("last_checked_date", "N/A"))
            try:
                last_checked_str = sealant_data.get("last_checked_date")
                if last_checked_str and last_checked_str != "N/A":
                    last_checked = datetime.datetime.strptime(last_checked_str, "%Y-%m-%d")
                    next_check_due_date = last_checked + datetime.timedelta(days=DEFAULT_SEALANT_CHECK_DAYS)
                    days_to_next_check = (next_check_due_date - datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)).days
                    sealant_ui['due_label'].config(text=f"Next Check: {days_to_next_check} days ({next_check_due_date.strftime('%Y-%m-%d')})", foreground="red" if days_to_next_check <= 0 else ("darkorange" if days_to_next_check <= YELLOW_THRESHOLD_DAYS else "forest green"))
                else: sealant_ui['due_label'].config(text="Next Check: N/A", foreground="black")
            except (ValueError, TypeError): sealant_ui['due_label'].config(text="Next Check: Invalid Date", foreground="black")

    def _create_log_panel(self, parent_frame):
        cols = ("Date", "Bike", "Item", "Odometer", "Notes"); self.log_tree = ttk.Treeview(parent_frame, columns=cols, show='headings', height=12)
        for col_name in cols: self.log_tree.heading(col_name, text=col_name); self.log_tree.column(col_name, width=140 if col_name != "Notes" else 250, anchor='w')
        vsb = ttk.Scrollbar(parent_frame, orient="vertical", command=self.log_tree.yview); hsb = ttk.Scrollbar(parent_frame, orient="horizontal", command=self.log_tree.xview)
        self.log_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set); vsb.pack(side='right', fill='y'); hsb.pack(side='bottom', fill='x'); self.log_tree.pack(expand=True, fill='both', pady=5)
        self._update_log_display()

    def _update_log_display(self):
        if hasattr(self, 'log_tree'):
            for item in self.log_tree.get_children(): self.log_tree.delete(item)
            for entry in self.data.get("maintenance_log", []): self.log_tree.insert("", "end", values=(entry.get("date", ""), entry.get("bike_name", ""), entry.get("item", ""), entry.get("odo_at_service", ""), entry.get("notes", "")))

    def _create_export_settings_panel(self, parent_frame):
        ttk.Button(parent_frame, text="Import Application Data (JSON)", command=self._import_app_data_from_json).pack(pady=10, padx=20, fill='x')
        ttk.Button(parent_frame, text="Export Maintenance Log to CSV", command=self._export_log_to_csv).pack(pady=10, padx=20, fill='x')

    def _import_app_data_from_json(self):
        filepath = filedialog.askopenfilename(defaultextension=".json", filetypes=[("JSON files", "*.json"), ("All files", "*.*")], title="Import Application Data from JSON")
        if not filepath: return
        try:
            with open(filepath, 'r') as f: loaded_data = json.load(f)
            if not isinstance(loaded_data, dict) or "bikes" not in loaded_data or "maintenance_log" not in loaded_data or not isinstance(loaded_data["bikes"], dict) or not isinstance(loaded_data["maintenance_log"], list):
                messagebox.showerror("Import Error", "Invalid data file structure."); return
            self.data = loaded_data; messagebox.showinfo("Import Successful", f"Data successfully imported from:\n{filepath}\nUI will be rebuilt.")
            self._create_all_bike_tabs(); self._update_all_displays(); self._set_default_tab(); self._save_all_data()
        except FileNotFoundError: messagebox.showerror("Import Error", f"File not found: {filepath}")
        except json.JSONDecodeError: messagebox.showerror("Import Error", f"Could not decode JSON: {filepath}.")
        except Exception as e: messagebox.showerror("Import Error", f"Unexpected error during import: {e}")

    def _export_log_to_csv(self):
        if not self.data.get("maintenance_log"): messagebox.showinfo("Export Log", "Log is empty."); return
        filepath = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")], title="Save Maintenance Log As")
        if not filepath: return
        try:
            with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
                fieldnames = ["date", "bike_name", "item", "odo_at_service", "notes"]
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames); writer.writeheader()
                for entry in self.data.get("maintenance_log", []): writer.writerow(entry)
            messagebox.showinfo("Export Successful", f"Log exported to:\n{filepath}")
        except IOError as e: messagebox.showerror("Export Error", f"Could not export log: {e}")

    def _save_all_data(self):
        try:
            for bike_id, tab_widgets_dict in self.bike_tabs_widgets.items():
                if bike_id in self.data["bikes"] and 'current_odo_entry' in tab_widgets_dict:
                    try: self.data["bikes"][bike_id]["current_odo"] = float(tab_widgets_dict['current_odo_entry'].get())
                    except ValueError: pass
            with open(self.data_file_path, 'w') as f: json.dump(self.data, f, indent=4)
        except IOError as e: messagebox.showwarning("Save Error", f"Could not save data to {self.data_file_path}: {e}")
        except Exception as e: messagebox.showwarning("Save Error", f"Unexpected error while saving: {e}")

    def _load_all_data(self):
        if self.data_file_path and os.path.exists(self.data_file_path):
            try:
                with open(self.data_file_path, 'r') as f:
                    loaded_data = json.load(f)
                    if not isinstance(loaded_data, dict) or "bikes" not in loaded_data or "maintenance_log" not in loaded_data or not isinstance(loaded_data["bikes"], dict) or not isinstance(loaded_data["maintenance_log"], list):
                        messagebox.showwarning("Load Error", "Data file invalid. Loading defaults."); return self._get_default_data_structure()
                    # Ensure hot_wax_count exists for all chains, add if missing (for backward compatibility)
                    for bike_id, bike_details in loaded_data.get("bikes", {}).items():
                        for chain in bike_details.get("chains", []):
                            if "hot_wax_count" not in chain:
                                chain["hot_wax_count"] = 0 # Default for old data
                    # Ensure brake pad structure is up-to-date
                    bike1_components = loaded_data.get("bikes", {}).get("bike1", {}).get("components", {})
                    if "brake_pads_disc" in bike1_components and "brake_pads_disc_front" not in bike1_components:
                        # Migrate old single disc brake pad entry
                        old_pad_odo = bike1_components["brake_pads_disc"].get("last_replaced_odo", 0.0)
                        bike1_components["brake_pads_disc_front"] = {"last_replaced_odo": old_pad_odo, "notes": ""}
                        bike1_components["brake_pads_disc_rear"] = {"last_replaced_odo": old_pad_odo, "notes": ""}
                        del bike1_components["brake_pads_disc"]
                        messagebox.showinfo("Data Migration", "Disc brake pad data has been migrated to separate Front/Rear entries for Nebula.")

                    return loaded_data
            except (IOError, json.JSONDecodeError) as e:
                messagebox.showwarning("Load Error", f"Could not load data from {self.data_file_path}: {e}. Loading defaults.")
                return self._get_default_data_structure()
        elif self.data_file_path:
            messagebox.showinfo("New Data File", f"Data file not found at {self.data_file_path}.\nA new one will be created.")
            return self._get_default_data_structure()
        else:
             messagebox.showerror("Critical Error", "Data file path not configured."); return self._get_default_data_structure()

    def _on_closing(self):
        self._save_all_data(); self.root.destroy()

if __name__ == "__main__":
    main_root = tk.Tk()
    app = AdvancedBikeMaintenanceApp(main_root)
    if hasattr(app, 'data_file_path') and app.data_file_path: main_root.mainloop()